### Linear Regression
1. Ordinary Least Squares Method: 
In this method, we find the regression coefficient weights that minimize the sum of the squared residuals.

Formula:  $$ weights = (X^T \cdot X)^{-1} \cdot X^T \cdot y$$

To find the predicted values, we multiply the feature matrix X with the weights vector.
Formula: $$ y_{pred} = X \cdot weights $$

Mean Squared Error: $$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_{pred} - y_{true})^2 $$

Importing required libraries and setting the profiler

In [135]:
# importing the libraries
import mxnet as mx
from mxnet import nd, autograd, gluon
import numpy as np
import LinearRegression_GPU as lr_gpu
import time
import pandas as pd
from mxnet import profiler
import re

In [136]:
# setting the context for the program 
mx.test_utils.list_gpus()
if mx.context.num_gpus() > 0:
    data_ctx = mx.gpu()
    model_ctx = mx.gpu()
else:
    data_ctx = mx.cpu()
    model_ctx = mx.cpu()

In [137]:
# setting the profiler for measuring the execution time and memory usage
profiler.set_config(profile_all=False,profile_symbolic = False, profile_imperative = False,profile_memory = True, profile_api = True, aggregate_stats=True,continuous_dump=False, filename='lin_reg_nf_gpu_profile.json')

Importing the dataset

In [138]:
# read data with pandas
data = pd.read_csv('train_data.csv')
# convert to numpy array
data = data.to_numpy()
test_data = pd.read_csv('test_data.csv')
# convert to numpy array
test_data = test_data.to_numpy()

Pre-processing step - Standardizing the data

In [139]:
# standardize the features
mean = np.mean(data, axis=0)
std = np.std(data, axis=0)
data = (data - mean) / std
# convert to NDArray
data = nd.array(data, ctx=data_ctx)

# standardize the features
mean = np.mean(test_data, axis=0)
std = np.std(test_data, axis=0)
test_data = (test_data - mean) / std
# convert to NDArray
test_data = nd.array(test_data, ctx=data_ctx)

c:\Users\Public\ipenv\envs\env1\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Splitting the dataset into train and test sets

In [140]:
# splitting into features and labels
features = nd.array(data[:, :-1], ctx=data_ctx)
labels = nd.array(data[:, -1], ctx=data_ctx)

In [141]:
X_train = features
y_train = labels
y_train = y_train.reshape((len(y_train), 1))

In [142]:
X_test = nd.array(test_data[:, :-1], ctx=data_ctx)
y_test = nd.array(test_data[:, -1], ctx=data_ctx)
y_test = y_test.reshape((len(y_test), 1))

In [143]:
# printing the shapes of the training set to check dimensions
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11917760, 12)
(11917760, 1)
(2979440, 12)
(2979440, 1)


Training the model

Marking the profiler for the training step

In [144]:
mx.nd.waitall() 

# starting the profiler
profiler.set_state('run')
start = time.time()

In [145]:
# training the model using OLS method
model = lr_gpu.LinearRegression()
# note the time before starting the training of the data
weights = model.OLS_fit(X_train, y_train)

Ending the profiler for the training step

In [146]:
# waiting for all operations to end, then stopping the profiler
mx.nd.waitall()
end = time.time()
profiler.set_state('stop')

In [147]:
results = profiler.dumps()

In [148]:
result = results
result = result.split('\n')

In [149]:
# splitting the result into a list of lists
for i in range(len(result)):
    result[i] = result[i].split()

Parsing the profiler data

In [150]:
# extracting the maximum gpu and cpu memory usage and the total execution time
max_gpu_use = 0
max_cpu_use = 0
total_execution_time = 0
# traversing over the lists and trying to find the maximum gpu and cpu memory usage and the total execution time
for i in result:
    if (len(i)>=1 and i[0]=='Memory:'):
        if (i[1]=='gpu/0'):
            max_gpu_use = float(i[-2])
        elif (i[1]=='cpu/0'):
            max_cpu_use = float(i[-2])
        else: continue
    # if the length of the list 6 and the second to sixth elements are numbers, then it is a time entry
    else:
        if (len(i)>=6):
            # if it is a valid time entry, then add it to the total execution time
            if (re.match(r'^-?\d+(?:\.\d+)$', i[-4]) is not None):
                total_execution_time += float(i[-4])

if (total_execution_time==0):
    total_execution_time = (end - start)*1000

Printing the result

In [151]:
print(f"Maximum training GPU memory usage: {max_gpu_use} KB")
print(f"Maximum training CPU memory usage: {max_cpu_use} KB")
print(f"Total training time: {total_execution_time} milli seconds (ms)")

Maximum training GPU memory usage: 1966430.375 KB
Maximum training CPU memory usage: 0 KB
Total training time: 3448.1371 milli seconds (ms)


Reseting the profiler for the prediction step

In [152]:
# set the profiler to default
profiler.set_config(profile_all=False,profile_symbolic = False, profile_imperative = False,profile_memory = False, profile_api = True, aggregate_stats=True,continuous_dump=False, filename='lin_reg_nf_gpu_profile.json')

In [153]:
mx.nd.waitall() 

# starting the profiler
profiler.set_state('run')
start = time.time()

In [154]:
# predicting the values
predictions = model.OLS_predict(X_test, weights)
# calculating the mean squared error
mse = nd.mean(((predictions - y_test) ** 2))

In [155]:
# prediction on the training set
predictions_train = model.OLS_predict(X_train, weights)
# calculating the mean squared error
mse_train = nd.mean(((predictions_train - y_train) ** 2))

In [156]:
# waiting for all operations to end, then stopping the profiler
mx.nd.waitall()
end = time.time()
profiler.set_state('stop')

In [157]:
results = profiler.dumps()

In [158]:
result = results
result = result.split('\n')

In [159]:
# splitting the result into a list of lists
for i in range(len(result)):
    result[i] = result[i].split() 

In [160]:
# parsing the profiler data
total_execution_time = 0
for i in result:
    if (len(i)>=6):
        if (re.match(r'^-?\d+(?:\.\d+)$', i[-4]) is not None):
            total_execution_time += float(i[-4])

if (total_execution_time==0):
    total_execution_time = (end - start)*1000

In [161]:
print(f"Total prediction/testing time: {total_execution_time} milli seconds (ms)")
print(f"Mean Squared Error on training set: {mse_train.asscalar()}")
print(f"Mean Squared Error on test set: {mse.asscalar()}")

Total prediction/testing time: 4208.5821000000005 milli seconds (ms)
Mean Squared Error on training set: 0.8208968639373779
Mean Squared Error on test set: nan
